<a href="https://colab.research.google.com/github/halfag/LocalLLM/blob/main/Mixtral_DnD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get update -q && apt install pciutils -y

!curl https://ollama.ai/install.sh | sh
!nohup ollama serve &

In [2]:
#%%capture
!ollama pull mixtral:8x7b

Streaming output truncated to the last 5000 lines.
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 79b7eca19f7a... 100% ▕▏   43 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 9dec05e9b2db... 100% ▕▏  484 B                         
verifying sha256 digest ⠦ pulling manifest 
pulling e9e56e8bb5f0... 100% ▕▏  26 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 79b7eca19f7a... 100% ▕▏   43 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 9dec05e9b2db... 100% ▕▏  484 B                         
verifying sha256 digest ⠧ pulling manifest 
pulling e9e56e8bb5f0... 100% ▕▏  26 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 79b7eca19f7a... 100% ▕▏   43 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 9dec05e9b2db... 100% 

In [3]:
!pip install crewai fastapi kaleido python-multipart uvicorn cohere tiktoken typing-extensions sec_api unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
from crewai import Agent, Task, Crew, Process
from langchain.agents import AgentType, initialize_agent, load_tools
#from langchain.llms import OpenAI

#os.environ["OPENAI_API_KEY"] = "Your Key"

from langchain.llms import Ollama
ollama_openhermes = Ollama(model="mixtral:8x7b")
# Pass Ollama Model to Agents: When creating your agents within the CrewAI framework, you can pass the Ollama model as an argument to the Agent constructor. For instance:

dm = Agent(
  role='Dungeon Master',
  goal='Host a short Dungeons and Dragons Session to do with raider goblins stealing the village food.',
  backstory="You're a world class Dungeon Master.",
#  tools=[
#    SearchTools.search_internet,
#    BrowserTools.scrape_and_summarize_website,
#  ],
  llm=ollama_openhermes, # Ollama model passed here
  verbose=False,
  allow_delegation=False
  # llm=OpenAI(temperature=0.7, model_name="gpt-4"). It uses langchain.chat_models, default is GPT4
)

ranger = Agent(
  role='ranger',
  goal='Role play your character. Be brave and lead the party to complete the quest.',
  backstory="You're a famous Level 3 Human Ranger, specialized on Hunting, Tracking, and Bowmanship.",
  llm=ollama_openhermes, # Ollama model passed here
  verbose=False,
  allow_delegation=False
)


fighter = Agent(
  role='fighter',
  goal='Role play your character and assist the party in completing the objective.',
  backstory="You're a newly recruited Level 1 Human Fighter with a shortsword.",
  llm=ollama_openhermes, # Ollama model passed here
  verbose=False,
  allow_delegation=False
)


# Create tasks for your agents
task1 = Task(description='Create a short Quest.', agent=dm)
task2 = Task(description='Explain the beginning scene of the quest. Only give details as the players unveil them.', agent=dm)

task3 = Task(description='Choose your action and the direction for the party.', agent=ranger)
task4 = Task(description='Choose your action and what you would like to do.', agent=fighter)
task5 = Task(description='Continue the quest accordingly making it unique and interesting.', agent=dm)
task6 = Task(description='If the players have successfully completed the quest, write "YOU WON".', agent=dm)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[dm, ranger, fighter],
  tasks=[task1, task2, task3, task4, task5, task6],
  verbose=True, # Crew verbose more will let you know what tasks are being worked on
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

# Get your crew to work!
result = crew.kickoff()
crew.tasks.remove(task1)
crew.tasks.remove(task2)
while "YOU WON" not in result:
  result = crew.kickoff()

Working Agent: Dungeon Master
Starting Task: Create a short Quest. ...
Task output: The village of Oakshade is being plagued by raider goblins who are stealing their food supplies. The village elder has sent out a call for brave adventurers to help them deal with this problem. The quest involves tracking down the goblin lair, rescuing any kidnapped villagers, and retrieving the stolen food. The adventure concludes in a final showdown with the goblin leader, where the players must make a choice: kill the goblins and risk further retaliation, or find a way to coexist peacefully.
Working Agent: Dungeon Master
Starting Task: Explain the beginning scene of the quest. Only give details as the players unveil them. ...
Task output: As the players arrive in the village of Oakshade, they are greeted by the panicked and worried faces of the villagers. The village elder approaches them, his face etched with concern. He explains that the village has been plagued by raider goblins who have been stea